# Session 2.1: BakeryAI - Introduction to Agents & Tools

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1aoazhS3U66qF64jXMTJhzQrguFTtAjX0?usp=sharing)

## 🎯 Session 2 Overview

Welcome to **Session 2**! Today we'll transform BakeryAI from a chatbot into an **intelligent agent** that can:

### What We'll Build Today:

✅ **Agents**: LLMs that can use tools to accomplish tasks  
✅ **Tool Integration**: Calendar, inventory, calculator, search  
✅ **ReAct Pattern**: Reasoning + Acting in loops  
✅ **Multi-Step Workflows**: Complex task decomposition  
✅ **LangGraph**: Stateful multi-agent systems  

### 🚀 BakeryAI Progress: Session 1 Complete → Adding Agency

**Session 1 Recap:**
- ✅ Basic chatbot with memory
- ✅ Semantic search
- ✅ Prompt templates
- ✅ Conversation management

**Day 2 Additions:**
- 🔧 Tools: Inventory check, delivery scheduling, payment
- 🤖 Agents: Decision-making and tool selection
- 🔄 Workflows: Multi-step order processing
- 📊 LangGraph: Complex agent coordination

## What is an Agent?

An **agent** is an LLM that can:
1. **Reason** about what needs to be done
2. **Choose** appropriate tools
3. **Execute** actions using those tools
4. **Observe** results
5. **Repeat** until task is complete

### Example Scenario:

**Customer**: "I want to order a chocolate cake for tomorrow at 2 PM"

**Agent Reasoning**:
1. "I need to check if chocolate cakes are in stock" → Use `inventory_check` tool
2. "I need to verify delivery availability for tomorrow 2 PM" → Use `check_delivery_slot` tool
3. "If available, calculate total with delivery fee" → Use `calculate_total` tool
4. "Confirm the order" → Use `place_order` tool

Let's build this! 🚀

In [1]:
!pip install -q langchain langchain-openai langgraph
!pip install -q pandas python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 9.9 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/IvanReznikov/mdx-langchain-conclave

Cloning into 'mdx-langchain-conclave'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 6), reused 24 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 240.64 KiB | 1.79 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [3]:
from google.colab import userdata
import os

# Set OpenAI API key from Google Colab's user environment or default
def set_openai_api_key(default_key: str = "YOUR_API_KEY") -> None:
    """Set the OpenAI API key from Google Colab's user environment or use a default value."""
    #if not (userdata.get("OPENAI_API_KEY") or "OPENAI_API_KEY" in os.environ):
    os.environ["OPENAI_API_KEY"] = userdata.get("MDX_OPENAI_API_KEY") or default_key


set_openai_api_key()
#set_openai_api_key("sk-...")

In [4]:
"""
# Enable LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("MDX_LANGSMITH_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "mdx-conclave-agents"  # Project name

print("✅ LangSmith Configuration:")
print(f"   Tracing Enabled: {os.getenv('LANGCHAIN_TRACING_V2')}")
print(f"   Project: {os.getenv('LANGCHAIN_PROJECT')}")
print("\n⚠️  Make sure to set LANGCHAIN_API_KEY in your .env file!")
"""

'\n# Enable LangSmith tracing\nos.environ["LANGCHAIN_TRACING_V2"] = "true"\nos.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"\nos.environ["LANGCHAIN_API_KEY"] = userdata.get("MDX_LANGSMITH_API_KEY")\nos.environ["LANGCHAIN_PROJECT"] = "mdx-conclave-agents"  # Project name\n\nprint("✅ LangSmith Configuration:")\nprint(f"   Tracing Enabled: {os.getenv(\'LANGCHAIN_TRACING_V2\')}")\nprint(f"   Project: {os.getenv(\'LANGCHAIN_PROJECT\')}")\nprint("\n⚠️  Make sure to set LANGCHAIN_API_KEY in your .env file!")\n'

In [5]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o") #for agent-use

In [6]:
# Load bakery data
try:
    cakes_df = pd.read_csv('/content/mdx-langchain-conclave/data/cake_descriptions.csv', encoding='cp1252')
    orders_df = pd.read_csv('/content/mdx-langchain-conclave/data/orders.csv', encoding='cp1252')
    prices_df = pd.read_csv('/content/mdx-langchain-conclave/data/price_list.csv', encoding='utf-16')
except FileNotFoundError:
    cakes_df = pd.DataFrame({
        'Name': ['Chocolate Truffle Cake', 'Vanilla Bean Cake', 'Red Velvet Cake'],
        'Category': ['Chocolate', 'Vanilla', 'Specialty'],
        'Available': [True, True, True],
        'Weight_grams': [1000, 1000, 1200],
        'Delivery_time_hr': [24, 24, 48]
    })

print("✅ BakeryAI Agent System Ready!")

✅ BakeryAI Agent System Ready!


## 1. Creating Your First Tool

Tools are functions that agents can call. Let's create a simple tool to check product availability.

In [7]:
from langchain_core.tools import tool

@tool
def check_product_availability(product_name: str) -> str:
    """Check if a product is available in inventory.

    Args:
        product_name: Name of the cake or product to check

    Returns:
        Availability status and details
    """
    # Search for product (case-insensitive)
    product = cakes_df[cakes_df['Name'].str.contains(product_name, na=False)]

    if product.empty:
        return f"Product '{product_name}' not found in our catalog."

    product_info = product.iloc[0]
    if product_info['Available']:
        return f"✅ {product_info['Name']} is available! Delivery time: {product_info['Delivery_time_hr']} hours."
    else:
        return f"❌ {product_info['Name']} is currently out of stock."

# Test the tool directly
print("Tool Test:")
print(check_product_availability.invoke({"product_name": "Chocolate"}))

Tool Test:
✅ Chocolate Truffle Cake is available! Delivery time: 12 hours.


## 2. Building More Tools for BakeryAI

In [8]:
cakes_df.head()

,Name,Category,Ingredients,Description,Energy_kcal,Weight_grams,Restrictions,Delivery_time_hr,Available
0,Torta della Nonna Amore,G,"Ricotta, pine nuts, lemon zest, vanilla, sugar...",A nostalgic Tuscan-inspired cake with creamy r...,320,850,"Contains dairy, gluten, eggs",24,Yes
1,Festiva della Sicilia,G,"Almonds, citrus zest, mascarpone, candied oran...",A zesty almond cake with mascarpone frosting a...,360,900,"Contains nuts, dairy, eggs",24,Yes
2,Dubai Midnight Pistachio Fantasy,G,"Pistachios, rose syrup, cardamom, white chocol...",Lush green cake with exotic floral pistachio n...,420,1000,"Contains nuts, dairy",36,Yes
3,Ferrari Redline Fudge,G,"Dark chocolate, red glaze, fudge, espresso syrup",Dense chocolate fudge cake with red mirror gla...,450,950,"Contains dairy, caffeine",24,Yes
4,Goalpost Delight - Soccer Fan Cake,G,"Vanilla sponge, fondant, jam, whipped cream","Themed cake with stadium design, perfect for f...",390,1200,May contain artificial colors,24,Yes


In [9]:
import random
from pydantic import BaseModel, Field
from langchain_core.tools.structured import StructuredTool

In [10]:
@tool
def calculate_order_total(product_name: str, quantity: int = 1) -> str:
    """Calculate the total cost including delivery fee.

    Args:
        product_name: Name of the product
        quantity: Number of items (default: 1)

    Returns:
        Total cost breakdown
    """
    try:
      filtered = prices_df[prices_df['Item'].str.lower().str.contains("carrot cake".lower(), na=False)]
      price = filtered['Unit_Price_AED'].iloc[0]
    except:
      price = 45  # default

    subtotal = price * quantity
    delivery_fee = 10 if subtotal < 100 else 0
    total = subtotal + delivery_fee

    return f"""Order Summary:
    - Product: {product_name} x {quantity}
    - Subtotal: {subtotal} AED
    - Delivery Fee: {delivery_fee} AED
    - Total: {total} AED"""

# Define Pydantic schema for the tool
class DeliverySlotInput(BaseModel):
    date: str = Field(description="Delivery date (e.g., 'tomorrow', '2025-10-23')")
    time: str = Field(description="Delivery time (e.g., '2 PM', '14:00')")

# Define the tool with @tool decorator and schema
@tool(args_schema=DeliverySlotInput)
def check_delivery_slot(date: str, time: str) -> str:
    """Check if a delivery slot is available.

    Args:
        date: Delivery date (e.g., 'tomorrow', '2025-10-23')
        time: Delivery time (e.g., '2 PM', '14:00')

    Returns:
        Slot availability status
    """
    # Parse "tomorrow"
    if date.lower() == 'tomorrow':
        delivery_date = datetime.now() + timedelta(days=1)
    else:
        delivery_date = datetime.now() + timedelta(days=2)

    # Simulate availability check
    is_available = random.random() < 0.9

    if is_available:
        return f"✅ Delivery slot available for {date} at {time}. We'll deliver between {time} and one hour after."
    else:
        return f"❌ Delivery slot for {date} at {time} is fully booked. Alternative times: 10 AM, 4 PM."


@tool
def get_product_recommendations(occasion: str, preferences: str = "") -> str:
    """Get product recommendations based on occasion and preferences.

    Args:
        occasion: Type of event (e.g., 'birthday', 'wedding', 'anniversary')
        preferences: Customer preferences (e.g., 'chocolate lover', 'vegan')

    Returns:
        Product recommendations
    """
    recommendations = []

    occasion_lower = occasion.lower()
    preferences_lower = preferences.lower()

    for _, product in cakes_df.iterrows():
        score = 0

        # Match based on occasion
        if 'birthday' in occasion_lower:
            if 'chocolate' in product['Category'].lower():
                score += 2
        elif 'wedding' in occasion_lower or 'anniversary' in occasion_lower:
            if 'vanilla' in product['Category'].lower() or 'specialty' in product['Category'].lower():
                score += 2

        # Match based on preferences
        if 'chocolate' in preferences_lower and 'chocolate' in product['Category'].lower():
            score += 3

        if score > 0:
            recommendations.append((product['Name'], score))

    # Sort by score
    recommendations.sort(key=lambda x: x[1], reverse=True)

    if recommendations:
        top_3 = recommendations[:3]
        result = f"Top recommendations for {occasion}:\n"
        for name, _ in top_3:
            result += f"  - {name}\n"
        return result
    else:
        return "We recommend our best-selling Chocolate Truffle Cake or Vanilla Bean Cake!"

# Test all tools
print("\n🔧 Testing Tools:\n")
print(calculate_order_total.invoke({"product_name": "Carrot Cake", "quantity": 2}))
print("\n" + check_delivery_slot.invoke({"date": "tomorrow", "time": "2 PM"}))
print("\n" + get_product_recommendations.invoke({"occasion": "birthday", "preferences": "chocolate lover"}))


🔧 Testing Tools:

Order Summary:
    - Product: Carrot Cake x 2
    - Subtotal: 290 AED
    - Delivery Fee: 0 AED
    - Total: 290 AED

✅ Delivery slot available for tomorrow at 2 PM. We'll deliver between 2 PM and one hour after.

We recommend our best-selling Chocolate Truffle Cake or Vanilla Bean Cake!


## 3. Creating a ReAct Agent

**ReAct** = **Rea**soning + **Act**ing

The agent will:
1. Think about what to do
2. Choose a tool
3. Execute the tool
4. Observe the result
5. Repeat until done

In [11]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

# Collect all tools
tools = [
    check_product_availability,
    calculate_order_total,
    check_delivery_slot,
    get_product_recommendations
]

# Get ReAct prompt template
# We'll use a simplified version for clarity
react_prompt = PromptTemplate.from_template("""
You are BakeryAI, an intelligent bakery assistant with access to tools.

Answer the customer's question by using tools when needed.
You have access to the following tools:

{tools}

Use this format:

Question: the input question you must answer
Thought: think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
""")

# Create the agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_prompt
)

# Create agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

print("✅ BakeryAI Agent Created!")

✅ BakeryAI Agent Created!


## 4. Testing the Agent

In [12]:
# Test 1: Simple availability check
print("Test 1: Product Availability\n")
print("=" * 70)

result = agent_executor.invoke({
    "input": "Is the Chocolate Truffle Cake available?"
})

print("\n📝 Final Answer:")
print(result['output'])

Test 1: Product Availability



> Entering new AgentExecutor chain...
Action: check_product_availability
Action Input: "Chocolate Truffle Cake"✅ Chocolate Truffle Cake is available! Delivery time: 12 hours.I now know the final answer.   
Final Answer: Yes, the Chocolate Truffle Cake is available. It requires a delivery time of 12 hours.

> Finished chain.

📝 Final Answer:
Yes, the Chocolate Truffle Cake is available. It requires a delivery time of 12 hours.


In [13]:
# Test 2: Multi-step reasoning
print("\n\nTest 2: Complex Order Query\n")
print("=" * 70)

result = agent_executor.invoke({
    "input": "I want to order 2 tiramisu cakes for tomorrow at 3 PM. How much will it cost and can you deliver?"
})

print("\n📝 Final Answer:")
print(result['output'])



Test 2: Complex Order Query



> Entering new AgentExecutor chain...
To answer this question, I need to check if the tiramisu cakes are available, calculate the total cost for 2 cakes, and check if there is a delivery slot available for tomorrow at 3 PM.

Action: check_product_availability
Action Input: "tiramisu cake"Product 'tiramisu cake' not found in our catalog.The tiramisu cake is not available in our inventory. I will check if there are any product recommendations for a similar occasion or based on popular preferences.

Action: get_product_recommendations
Action Input: "dessert", "tiramisu"We recommend our best-selling Chocolate Truffle Cake or Vanilla Bean Cake!The tiramisu cake is currently not available in our inventory. As an alternative, I recommend considering our best-selling Chocolate Truffle Cake or Vanilla Bean Cake. 

Please let me know if you'd like more information about these options or if you want to proceed with one of them.Invalid Format: Missing 'Action:' aft

In [14]:
# Test 3: Recommendation
print("\n\nTest 3: Recommendation Request\n")
print("=" * 70)

result = agent_executor.invoke({
    "input": "What cake would you recommend for a birthday party? I love chocolate!"
})

print("\n📝 Final Answer:")
print(result['output'])



Test 3: Recommendation Request



> Entering new AgentExecutor chain...
Since the customer is looking for a cake recommendation for a birthday party and mentioned a preference for chocolate, I should check for product recommendations based on the occasion and preferences.

Action: get_product_recommendations
Action Input: "birthday", "chocolate lover"We recommend our best-selling Chocolate Truffle Cake or Vanilla Bean Cake!Both the Chocolate Truffle Cake and the Vanilla Bean Cake are excellent options for a birthday party, especially for a chocolate enthusiast.

Final Answer: For a birthday party, I recommend our best-selling Chocolate Truffle Cake, which is perfect for chocolate lovers! Alternatively, you might also consider the Vanilla Bean Cake if you'd like to try something different.

> Finished chain.

📝 Final Answer:
For a birthday party, I recommend our best-selling Chocolate Truffle Cake, which is perfect for chocolate lovers! Alternatively, you might also consider the Vanil

## 5. Understanding the ReAct Loop

Let's trace through what happened in the agent's reasoning:

In [15]:
# Detailed trace
print("""
📊 ReAct Loop Breakdown:

Customer: "I want to order 2 chocolate cakes for tomorrow at 3 PM. How much will it cost and can you deliver?"

Step 1:
  Thought: "I need to check if chocolate cakes are available"
  Action: check_product_availability
  Action Input: "Chocolate Truffle Cake"
  Observation: "✅ Chocolate Truffle Cake is available! Delivery time: 24 hours."

Step 2:
  Thought: "I need to calculate the cost for 2 cakes"
  Action: calculate_order_total
  Action Input: {"product_name": "Chocolate Truffle Cake", "quantity": 2}
  Observation: "Subtotal: 90 AED, Delivery Fee: 0 AED, Total: 90 AED"

Step 3:
  Thought: "I need to check if delivery is available tomorrow at 3 PM"
  Action: check_delivery_slot
  Action Input: {"date": "tomorrow", "time": "3 PM"}
  Observation: "✅ Delivery slot available for tomorrow at 3 PM"

Step 4:
  Thought: "I now have all the information needed"
  Final Answer: "Yes! We can deliver 2 Chocolate Truffle Cakes tomorrow at 3 PM. Total cost: 90 AED..."

✅ The agent autonomously decided which tools to use and in what order!
""")


📊 ReAct Loop Breakdown:

Customer: "I want to order 2 chocolate cakes for tomorrow at 3 PM. How much will it cost and can you deliver?"

Step 1:
  Thought: "I need to check if chocolate cakes are available"
  Action: check_product_availability
  Action Input: "Chocolate Truffle Cake"
  Observation: "✅ Chocolate Truffle Cake is available! Delivery time: 24 hours."

Step 2:
  Thought: "I need to calculate the cost for 2 cakes"
  Action: calculate_order_total
  Action Input: {"product_name": "Chocolate Truffle Cake", "quantity": 2}
  Observation: "Subtotal: 90 AED, Delivery Fee: 0 AED, Total: 90 AED"

Step 3:
  Thought: "I need to check if delivery is available tomorrow at 3 PM"
  Action: check_delivery_slot
  Action Input: {"date": "tomorrow", "time": "3 PM"}
  Observation: "✅ Delivery slot available for tomorrow at 3 PM"

Step 4:
  Thought: "I now have all the information needed"
  Final Answer: "Yes! We can deliver 2 Chocolate Truffle Cakes tomorrow at 3 PM. Total cost: 90 AED..."

✅ 

## 6. Agent with Error Handling

In [16]:
# Test with unavailable product
print("Test 4: Handling Unavailable Products\n")
print("=" * 70)

result = agent_executor.invoke({
    "input": "I want to order a Unicorn Cake for tomorrow"
})

print("\n📝 Final Answer:")
print(result['output'])

print("\n✅ Agent gracefully handled non-existent product!")

Test 4: Handling Unavailable Products



> Entering new AgentExecutor chain...
I need to check if the Unicorn Cake is available in our inventory first. 

Action: check_product_availability
Action Input: "Unicorn Cake"
Product 'Unicorn Cake"
' not found in our catalog.It seems that there might be a typo or the product isn't listed as "Unicorn Cake" in our catalog. Let's try to find an alternative name or get recommendations for cakes suitable for celebrations.

Action: get_product_recommendations
Action Input: "birthday", "unicorn theme"
We recommend our best-selling Chocolate Truffle Cake or Vanilla Bean Cake!It seems we don't have a product named "Unicorn Cake" in our catalog, but we do recommend our best-selling Chocolate Truffle Cake and Vanilla Bean Cake, which are popular choices for celebrations with a unicorn theme. Would you like to check the availability or the total cost for either of these options?Invalid Format: Missing 'Action:' after 'Thought:'It seems we don't have a pro

## 7. Streaming Agent Responses

Show intermediate steps in real-time.

In [17]:
# Stream agent responses
print("Streaming Agent Execution:\n")

for step in agent_executor.stream(
    {"input": "Check if chocolate cake is available and give me the price for 3 cakes"}
):
    if 'actions' in step:
        for action in step['actions']:
            print(f"🔧 Tool: {action.tool}")
            print(f"📥 Input: {action.tool_input}")

    if 'steps' in step:
        for s in step['steps']:
            print(f"📤 Output: {s.observation[:100]}...\n")

    if 'output' in step:
        print(f"\n✅ Final: {step['output']}")

Streaming Agent Execution:



> Entering new None chain...
🔧 Tool: check_product_availability
📥 Input: chocolate cake
To answer the question, I first need to check if the chocolate cake is available in the inventory. Following that, if it is available, I'll calculate the total cost for 3 cakes. 

Action: check_product_availability
Action Input: "chocolate cake"Product 'chocolate cake' not found in our catalog.📤 Output: Product 'chocolate cake' not found in our catalog....

🔧 Tool: get_product_recommendations
📥 Input: birthday", "chocolate lover
The chocolate cake is not available in our catalog. I should provide an alternative recommendation and calculate the price for a similar product instead.

Action: get_product_recommendations
Action Input: "birthday", "chocolate lover"We recommend our best-selling Chocolate Truffle Cake or Vanilla Bean Cake!📤 Output: We recommend our best-selling Chocolate Truffle Cake or Vanilla Bean Cake!...

🔧 Tool: calculate_order_total
📥 Input: Chocolate Tru

## 🎯 Exercise 1: Add More Tools

**Task**: Create these additional tools for BakeryAI:
1. `cancel_order(order_id: str)` - Cancel an existing order
2. `track_order(order_id: str)` - Get order status
3. `apply_discount_code(code: str, total: float)` - Apply promotional codes
4. `get_customer_loyalty_points(customer_id: str)` - Check loyalty points

In [18]:
@tool
def cancel_order(order_id: str) -> str:
    """Cancel an existing order.

    Args:
        order_id: The order ID to cancel

    Returns:
        Cancellation status
    """
    # TODO: Implement order cancellation logic
    pass

@tool
def track_order(order_id: str) -> str:
    """Track the status of an order.

    Args:
        order_id: The order ID to track

    Returns:
        Current order status
    """
    # TODO: Implement order tracking
    pass

# Add more tools here
# Then test your agent with the new tools

## 🎯 Exercise 2: Build a Conversational Agent with Memory

**Task**: Enhance the agent to remember previous interactions:
1. Keep track of what the customer has asked
2. Remember products they've shown interest in
3. Don't ask for information already provided

In [19]:
from langchain.memory import ConversationBufferMemory

# TODO: Create agent with memory
# Hint: Use ConversationBufferMemory with AgentExecutor

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# agent_with_memory = AgentExecutor(..., memory=memory)

## Summary: What We Built

### ✅ Session 2.1 Achievements:

1. **Understanding Agents**: LLMs that can use tools autonomously
2. **Created Tools**: 4 custom tools for BakeryAI
3. **ReAct Pattern**: Reasoning + Acting loop
4. **Agent Executor**: Orchestrates tool usage
5. **Error Handling**: Graceful failures
6. **Streaming**: Real-time agent execution

### 🔧 BakeryAI Tools Now Available:

✨ `check_product_availability` - Inventory checks  
✨ `calculate_order_total` - Pricing calculations  
✨ `check_delivery_slot` - Scheduling  
✨ `get_product_recommendations` - Smart suggestions  

### 🚀 Next: Notebook 2.2

We'll build **more complex tools** including:
- Database integration (customer lookup)
- External API calls (payment processing)
- File operations (order receipts)
- Advanced error handling and retries